---
title: "Statistical Process Control"
format: html
---

# 🏭 Process Data Analysis & Monitoring
**Portfolio Project 6 — Statistical Process Control (SPC)**

---

## Objective
Implement control charts, capability indices, and drift detection
on simulated manufacturing process data.

## Dataset
**Simulated CNC / Chemical Reactor Process** (mirrors Tennessee Eastman benchmark structure)
Tennessee Eastman: https://github.com/Ramin-Khalatbari/dataset-Tennessee-Eastman

---

In [ ]:
# 1. Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.signal import find_peaks
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print('Imports OK')

## 1. Synthetic Process Data

In [ ]:
# 2. Generate process streams with injected faults
def gen_process_data(n=2000, seed=55):
    rng = np.random.default_rng(seed)

    # Normal-state streams
    temp = rng.normal(85.0, 1.5, n)   # reactor temperature
    pressure = rng.normal(3.2, 0.2, n)    # reactor pressure
    flow = rng.normal(50.0, 3.0, n)   # feed flow
    conc_A = rng.normal(0.45, 0.03, n)  # reactant A concentration
    conc_B = 1.0 - conc_A + rng.normal(0, 0.01, n)  # reactant B (mass balance)

    # Inject faults
    fault_label = np.zeros(n, dtype=int)

    # Fault 1: Temperature drift (gradual) — samples 500-700
    drift = np.linspace(0, 4, 200)
    temp[500:700] += drift
    fault_label[500:700] = 1

    # Fault 2: Pressure spike — samples 1000-1020
    temp[1000:1020] += rng.normal(0, 0.5, 20)
    pressure[1000:1020] += rng.uniform(1.0, 2.0, 20)
    fault_label[1000:1020] = 2

    # Fault 3: Flow step change — samples 1500-1600
    flow[1500:1600] += 8
    conc_A[1500:1600] -= 0.05  # downstream effect
    fault_label[1500:1600] = 3

    df = pd.DataFrame({
        'Temperature': temp.round(2),
        'Pressure': pressure.round(3),
        'Flow': flow.round(2),
        'Conc_A': conc_A.round(4),
        'Conc_B': conc_B.round(4),
        'Fault': fault_label
    })
    df['Time'] = pd.date_range('2024-06-01', periods=n, freq='5min')
    return df


df = gen_process_data()
print(df.shape)
df.head()

## 2. Control Charts (Xbar-S)

In [ ]:
# 3. Individual + Moving Range (I-MR) control chart for Temperature
# Using first 500 samples (normal phase) to set limits
normal = df.iloc[:500]
mean_T = normal['Temperature'].mean()
std_T = normal['Temperature'].std()

UCL = mean_T + 3*std_T
LCL = mean_T - 3*std_T

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(df['Time'], df['Temperature'], lw=0.8,
        color='steelblue', label='Temperature')
ax.axhline(UCL, color='red',   ls='--', lw=1.2, label=f'UCL = {UCL:.2f}')
ax.axhline(LCL, color='red',   ls='--', lw=1.2, label=f'LCL = {LCL:.2f}')
ax.axhline(mean_T, color='green', ls='-', lw=1.0, label=f'CL = {mean_T:.2f}')

# Highlight violations
violations = (df['Temperature'] > UCL) | (df['Temperature'] < LCL)
ax.scatter(df.loc[violations, 'Time'], df.loc[violations, 'Temperature'],
           s=30, color='red', zorder=5, label=f'Violations ({violations.sum()})')

ax.set_title('I-Chart — Reactor Temperature', fontsize=13)
ax.set_ylabel('Temperature (°C)')
ax.set_xlabel('Time')
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# 4. Control charts for all variables
vars_to_plot = ['Temperature', 'Pressure', 'Flow', 'Conc_A']

fig, axes = plt.subplots(len(vars_to_plot), 1, figsize=(16, 12), sharex=True)

for ax, var in zip(axes, vars_to_plot):
    norm_data = df[var].iloc[:500]
    mu = norm_data.mean()
    sig = norm_data.std()
    ucl = mu + 3*sig
    lcl = mu - 3*sig

    ax.plot(df['Time'], df[var], lw=0.7, color='steelblue')
    ax.axhline(ucl, color='red',   ls='--', lw=1)
    ax.axhline(lcl, color='red',   ls='--', lw=1)
    ax.axhline(mu,  color='green', ls='-',  lw=0.8)
    ax.set_ylabel(var, fontsize=10)

    # Shade fault regions
    for f_id, color in [(1, '#ff9999'), (2, '#ffcc99'), (3, '#cc99ff')]:
        mask = df['Fault'] == f_id
        if mask.any():
            ax.axvspan(df.loc[mask, 'Time'].iloc[0], df.loc[mask, 'Time'].iloc[-1],
                       alpha=0.15, color=color, label=f'Fault {f_id}')
    if var == vars_to_plot[0]:
        ax.legend(loc='upper left', fontsize=8)

axes[-1].set_xlabel('Time')
plt.suptitle('Multi-Variable Control Charts with Fault Windows',
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 3. Process Capability Analysis

In [ ]:
# 5. Cpk calculation for each variable (normal phase only)
def cpk(data, usl, lsl):
    mu = data.mean()
    sig = data.std()
    cpu = (usl - mu) / (3*sig) if sig > 0 else np.inf
    cpl = (mu - lsl) / (3*sig) if sig > 0 else np.inf
    return min(cpu, cpl)


# Define spec limits (± 4 sigma for illustration)
specs = {}
for var in vars_to_plot:
    mu = df[var].iloc[:500].mean()
    sig = df[var].iloc[:500].std()
    specs[var] = {'USL': mu + 4*sig, 'LSL': mu - 4*sig,
                  'Cpk': cpk(df[var].iloc[:500], mu+4*sig, mu-4*sig)}

specs_df = pd.DataFrame(specs).T
print('Process Capability (Normal Phase):')
print(specs_df.round(4))

fig, ax = plt.subplots(figsize=(8, 4))
colors = ['green' if c >= 1.33 else 'orange' if c >=
          1.0 else 'red' for c in specs_df['Cpk']]
specs_df['Cpk'].plot(kind='bar', ax=ax, color=colors, edgecolor='white')
ax.axhline(1.33, color='green', ls='--', lw=1, label='Target Cpk=1.33')
ax.axhline(1.0,  color='orange', ls='--', lw=1, label='Min Cpk=1.0')
ax.set_title('Process Capability Index (Cpk)')
ax.set_ylabel('Cpk')
ax.legend()
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

## 4. CUSUM Drift Detection

In [ ]:
# 6. CUSUM chart for Temperature
k = 0.5 * std_T   # allowance
h = 5.0 * std_T   # decision interval

cusum_pos = np.zeros(len(df))
cusum_neg = np.zeros(len(df))
alerts = np.zeros(len(df), dtype=bool)

for i in range(1, len(df)):
    cusum_pos[i] = max(0, cusum_pos[i-1] +
                       (df['Temperature'].iloc[i] - mean_T) - k)
    cusum_neg[i] = max(0, cusum_neg[i-1] -
                       (df['Temperature'].iloc[i] - mean_T) - k)
    alerts[i] = (cusum_pos[i] > h) or (cusum_neg[i] > h)

fig, axes = plt.subplots(2, 1, figsize=(16, 7), sharex=True)

axes[0].plot(df['Time'], cusum_pos, lw=1, color='steelblue', label='CUSUM+')
axes[0].plot(df['Time'], -cusum_neg, lw=1, color='orange', label='CUSUM−')
axes[0].axhline(h,  color='red', ls='--', lw=1, label=f'H = {h:.2f}')
axes[0].axhline(-h, color='red', ls='--', lw=1)
axes[0].set_title('CUSUM Chart — Temperature')
axes[0].set_ylabel('CUSUM Value')
axes[0].legend()

# Alert indicator
axes[1].fill_between(df['Time'], alerts.astype(int), color='red', alpha=0.5)
axes[1].set_title('CUSUM Alert Signal')
axes[1].set_ylabel('Alert')
axes[1].set_xlabel('Time')
axes[1].set_yticks([0, 1])
axes[1].set_yticklabels(['No Alert', 'Alert'])

plt.tight_layout()
plt.show()
print(f'Total CUSUM alert samples: {alerts.sum()}')

## Summary
- Implemented I-MR control charts for 4 process variables
- Detected all three injected faults via limit violations
- Calculated Cpk for normal-phase capability assessment
- CUSUM successfully detected the gradual temperature drift early